# 0. Loading libraries and Classes

In [0]:
# Import necessary libraries for this notebook
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from IPython.display import Image

# Import train_test_split to separate train and test set
from sklearn.model_selection import train_test_split
# Import VarianceThreshold to removes all low-variance features
from sklearn.feature_selection import VarianceThreshold

%matplotlib inline

An **important** thing to remember is that we should perform feature selection in conjunction with the model selection

# 1. Non-data columns
Columns often used for identifying observations typically lack informational value.
Such columns include unique identifiers like ID columns, telephone numbers, and email addresses.

We will be working with the subset of Santander Bank dataset \\(^{1}\\) (30 000 rows), which contain anonymized features to predict customer satisfaction regarding their experience with the bank.

In [0]:
# Load the subset dataset called 'subset_santander.csv' and store it to variable data
data = pd.read_csv('../../Data/subset_santander.csv')

# Print the shape of the dataframe and get the first 10 rows
print(data.shape)
data.head(10)

Feature selection should be performed only on our training data to avoid overfitting. Let's split our dataset and drop our target feature 'TARGET'.

In [0]:
# Separate dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels = ['TARGET'], axis = 1),
                                                    data['TARGET'],
                                                    test_size = 0.3,
                                                    random_state = 42)

# Get the shape of training and testing set
X_train.shape, X_test.shape

If we have a look at the table above we see that there is a column called `ID` which we can drop since we must not use it for predictions.
This task has to be done manually, since there is no way of inferring this information from the data.

In [0]:
# Drop the ID column
X_train.drop("ID", axis=1, inplace=True)
X_test.drop("ID", axis=1, inplace=True)

# Get the shape after removing the ID column
X_train.shape, X_test.shape

# 2. Constant Features
Constant features do not provide any information useful for further analysis or predicting the target variable. These features provide only a single value for all of the observations in the dataset. Therefore, we can remove them from the dataset.

There are 369 features in our dataset. 
We can now look at whether there are some constant features in `X_train` set using the
[`.var()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.var.html)
method which computes the variance along the columns.
Within this function we can specify argument `ddof = 1`.
For more information see the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.var.html).

In [0]:
# Get the features that have the variance equal to zero
# Optional: Specify ddof = 1 within the `.var()` function 

our_constant_features = X_train.loc[:, X_train.var(ddof = 1) == 0]

In [0]:
# Print our_constant_features
our_constant_features

There are 64 features with zero variance, which will be removed from our dataset. 

In [0]:
# Remove constant features from X_train, do not forget specify argument inplace = True
X_train.drop(labels= our_constant_features, axis=1, inplace=True)
# Remove constant features from X_test, do not forget specify argument inplace = True
X_test.drop(labels= our_constant_features, axis=1, inplace=True)

# Get the shape after removing constant features
X_train.shape, X_test.shape

# 3. Quasi-constant Features

Quasi-constant features have very low variance (close to 0) and contain little information, which is not useful for us. These approximately constant features won't help the ML model's performance, therefore we should consider removing them. 

We could filter quasi-constant features with Pandas in a similar way as we did with constant features, with one difference - we would set a specific threshold. Nevertheless, now we'll leave Pandas behind and rather use scikit learn which offers a more convenient way to find quasi-constant features. 

In the
[`sklearn.feature_selection`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection)
module we can find a feature selector called
[`VarianceThreshold()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html),
which finds all features with low variance (based on a specified threshold) and removes them. 
You can find more information about this selector [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html).

As a first step, we define our selector for quasi-constant features with a threshold of 0.01. In other words, this is the minimum value of the variance we want to have in the dataset. 

In [0]:
# Define VarianceThreshold() object and specify parameter threshold = 0.01
our_selector = VarianceThreshold(threshold = 0.01)

Next, we fit `our_selector` with the `X_train` data to find quasi-constant features.

In [0]:
# Fit X_train with our_selector
our_selector.fit(X_train)

Using
[`get_support()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold.get_support)
method, we can get all of the features we want to keep along with their names.
This _mask_ we will use later to assign names to columns.

*Note: You might wonder why we are saving the feature names in the `features_to_keep` variable. Scikit learn will always save the necessary state inside of the fitted transformer.
However, in this example we only do this for our convience, so that we can later on go back from the nameless Numpy array to a nice dataframe with all the column names.*

In [0]:
# Get the mask of features we want to keep in the dataset
features_to_keep = X_train.columns[our_selector.get_support()]

# Print the length
print('The number of features that will be kept: {}'.format(len(features_to_keep)))

The next step is to transform `X_train` and `X_test` using `our_selector`.

In [0]:
# Transform X_train and X_test = in this step, the quasi-constant featues will be finally removed
X_train = our_selector.transform(X_train)
X_test = our_selector.transform(X_test)

# Get the shape of X_train and X_test
X_train.shape, X_test.shape

After the trnsformation `X_train` is a
[`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html)
object and needs to be transformed into a Pandas DataFrame again.
Here, we use our created `features_to_keep` variable to assign column names.

In [0]:
# Print X_train
X_train

In [0]:
# Convert X_train to a Pandas DataFrame
X_train= pd.DataFrame(X_train)
# Using the '.columns' attribute assign column names
X_train.columns = features_to_keep

# Convert X_test to a Pandas DataFrame
X_test= pd.DataFrame(X_test)
# Using the '.columns' attribute assign column names
X_test.columns = features_to_keep

# Get the first 5 rows of X_train
X_train.head()

# 4. Duplicated Features (READ-ONLY)
Duplicated features are totally redundant features, thus not providing any useful or new information for improving the model's performance.

To better understand how duplicated features can be treated using Pandas we create new DataFrame.
We've already seen the [`duplicated()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html)
function which returns a boolean Series denoting duplicate rows.
To identify duplicated features, we have to first transpose our data frame, in other words, we swap the rows and columns.
More information [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transpose.html).

You might wonder again, why are we not using the
[`scikit-learn`](https://scikit-learn.org/stable/index.html)? The reason is that **duplicated features should be already addressed within data integration and preprocessing**. You might remember that these are the earliest stages. The reason is that duplicated values usually occur when we are merging data from various sources. It was not a priority for scikit developers to implement a specific transformer for this.

We are doing an ugly operation of swapping rows and columns to make use of Pandas functionality and make this operation as easy as possible. Yes, we are only advising you to do this with a small dataset. If you have a *big* dataset, for example counted in TBs, you should not and most likely will not be able to do this.

In [0]:
# Run this code to create new DataFrame
our_data  =[(1, 'Colin Trevorrow', 124, 150 ,'Colin Trevorrow', 150, 'Jurassic World'),
           (2, 'George Miller', 120, 55, 'George Miller', 55, 'Mad Max: Fury Road'),
           (3, 'Robert Schwentke', 119, 112, 'Robert Schwentke', 112, 'Insurgent'),
           (4, 'J.J. Abrams', 136, 220,'J.J. Abrams', 220, 'Star Wars: The Force Awakens'),
           (5, 'James Wan', 137, 154, 'James Wan', 154, 'Furious 7'),
           (6, 'Bruce Brown', 95, 25, 'Bruce Brown', 25, 'The Endless Summer'),
           (7, 'Woody Allen', 80, 15, 'Woody Allen', 15, 'What`s Up, Tiger Lily?'),
           (8, 'James Cameron', 162, 180, 'James Cameron', 180, 'Avatar'),
           (9, 'Carl Tibbetts', 74, 44, 'Carl Tibbetts', 44, 'Black Mirror: White Christmas'),
           (10, 'Harold P. Warren', 74, 8, 'Harold P. Warren', 8, 'Manos: The Hands of Fate')]

movies = pd.DataFrame(our_data, columns= ['id', 'director', 'runtime','total_votes', 'name', 'number_of_votes', 'title'])

In [0]:
# Print movies DataFrame
movies

In [0]:
# Print the shape of movies
movies.shape

As we can see, the movies DataFrame contains 10 rows and 7 features. 
Now we use the 
[`.transpose()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transpose.html)
method.

In [0]:
# Transpose movies and store it to variable movies_transpose
movies_transpose = movies.transpose()
movies_transpose

In [0]:
# Get the shape of movies_transpose
movies_transpose.shape

After transposing, there are 7 rows (features) and 10 columns in `movies_transpose`.

Now we apply chained
[`duplicated().sum()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html)
function on `movies_transpose` that give us the total number of duplicated rows (features).

In [0]:
# Get the total number of duplicated rows (features)
movies_transpose.duplicated().sum()

There are 2 duplicated rows (features), containing the same observations.
We can drop duplicated rows using
[`.drop_duplicates()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html).
By setting `keep = 'first'` parameter, we determine which duplicated row we want to keep.

In [0]:
# Drop duplicates and store the result in the variable unique_features
unique_features = movies_transpose.drop_duplicates(keep = 'first').transpose()

In [0]:
# Get duplicated features and store the result in the variable duplicated_feature
duplicated_features = [column for column in movies.columns if column not in unique_features]
duplicated_features

In [0]:
# Drop the duplicated features from the original DataFrame
movies.drop(labels = duplicated_features, axis = 1, inplace = True)
movies

However, this approach is not computationally and memory-efficient if you have a really large DataFrame with thousands of rows.
As
[`scikit learn`](https://scikit-learn.org/stable/index.html)
does not offer a method to handle duplicated features, we need to create some function for this purpose.
Then we drop duplicated features using Pandas'
[`.drop()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html)
method.

In [0]:
# Create an empty list for duplicated features
features_duplicates = []

# Create a for loop for iterating over the range of columns from the X_train set
for col in range(len(X_train.columns)):
    column_1 = X_train.columns[col]
    # Find duplicated features by comparing columns using .equals
    for column_2 in X_train.columns[col + 1:]:
        if X_train[column_1].equals(X_train[column_2]):
            features_duplicates.append(column_2)
            
len(features_duplicates)

In [0]:
# Drop duplicated features from X_train and X_test
X_train.drop(labels = features_duplicates, axis = 1, inplace = True)
X_test.drop( labels = features_duplicates, axis = 1, inplace = True)

In [0]:
# Get the shape of X_train and X_test
X_train.shape, X_test.shape

[`scikit learn`](https://scikit-learn.org/stable/index.html)
module offers many methods such as selecting features based on their importance but we will not go there.
You can find these methods in the
[documentation](https://scikit-learn.org/stable/modules/feature_selection.html).
Now we'll look at the correlation between features.

# 5. Correlation
Features with high correlation have almost the same effect on the target feature.
We can visualize relationships between features using
[`.corr()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)
method to understand the data better.

In [0]:
# Find the correlation among the columns and store it in variable correlation_matrix
correlation_matrix = X_train.corr()

In [0]:
# Plot the correlation matrix
plt.figure(figsize=(11, 11))
sns.heatmap(correlation_matrix, cmap="vlag", vmin=-1, vmax=1);

We'll find the highly correlated features using a function based on correlation coefficients above the threshold of 0.8.

In [0]:
def correlation(dataset, threshold):
    # Create set for correlated columns
    corelated_cols = set()  
    # Compute correlation 
    corr_matrix = dataset.corr()
    for c in range(len(corr_matrix.columns)):
        for j in range(c):
            # Take absolute correlation coefficient value 
            # If abs values are above threshold ...
            if abs(corr_matrix.iloc[c, j]) > threshold: 
                # ... Get name of column
                colname = corr_matrix.columns[c]
                corelated_cols.add(colname)
    return corelated_cols

In [0]:
# Use correlation function on X_train with threshold 0.8
corr_features_to_drop = correlation(X_train, 0.8)
len(set(corr_features_to_drop))

In [0]:
# Drop correlated features from X_train and X_test
X_train.drop(labels = corr_features_to_drop, axis = 1, inplace = True)
X_test.drop(labels = corr_features_to_drop, axis = 1, inplace = True)

In [0]:
# Get the shape of X_train and X_test
X_train.shape, X_test.shape

## TASK

You will be using an altered dataset containing variants of the Portuguese 'Vinho Verde' wine \\(^{2}\\). The features provide information about wine samples recorded based on physicochemical tests. There is also the target feature that denotes the quality score of the sample. 

In [0]:
# Load the dataset 'wine_quality.csv' and store it to variable wine
wine = pd.read_csv('../../Data/quality_of_wine.csv', sep = ',', )
# Get the first 10 rows
wine.head(10)

In [0]:
# Print the dataframe's datatypes
wine.dtypes

Several numerical features are stored as float or integer, and one feature is stored as a string in our dataset. 

These numerical variables can be used to predict the quality of the wine samples. So the **'quality'** column is our **target feature**.  

In [0]:
# Separate the dataset 'wine' into training and testing set
# Store it in variables: X_training, X_testing, y_training, y_testing
# Drop the target feature 'quality'
# Set test_size = 0.3 and random_state = 42

X_training, X_testing, y_training, y_testing = train_test_split(wine.drop(labels = ['quality'], axis=1), 
                                                                wine['quality'],
                                                                test_size = 0.3,
                                                                random_state = 42)

# Get the shape of training and testing set
X_training.shape, X_testing.shape

This data set also contains an ID column named 'Unnamed: 0'.
Since it does not contain any information we will drop it.

In [0]:
# Drop ID column
X_training.drop("Unnamed: 0", axis=1, inplace=True)
X_testing.drop("Unnamed: 0", axis=1, inplace=True)

As we already know there is one non-numerical variable ('type'). Let's look at the unique values of this feature.

In [0]:
# Print unique values of 'type' column in X_training and X_testing sets
print(X_training['type'].unique())
print(X_testing['type'].unique())

The datasets can also contain this type of constant feature stored as a string and have only one unique value/category. As this variable is not really helpful, we will drop it from the dataset. 

In [0]:
# TASK 1 >>>> Remove the constant feature from X_training using '.drop()'. Do not forget to specify the argument inplace = True
X_training.drop('type', axis = 1, inplace = True)

# TASK 2 >>>> Remove the constant feature from X_testing using '.drop()'. Do not forget to specify the argument inplace = True
X_testing.drop('type', axis = 1, inplace = True)

# Get the shape of X_training and X_testing sets
X_training.shape, X_testing.shape

Now we want to select only those features that have a variance above the threshold = 0.01. Again, we will find quasi-constant features using scikit learn's [`VarianceThreshold`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html) as we did in the previous example.

In [0]:
# TASK 3 >>>> Define a VarianceThreshold() object, specify the parameter threshold = 0.01 and store it in variable 'selector'
selector = VarianceThreshold(0.01)

In [0]:
# TASK 4 >>>> Fit X_training with 'selector'
selector.fit(X_training)

In [0]:
# Get a mask of those features we want to keep in the dataset and store it in the variable 'features_we_keep'
features_we_keep = X_training.columns[selector.get_support()]
# Print the length of the variable features_we_keep
print('The number of features that will be kept: {}'.format(len(features_we_keep)))

In [0]:
# Print the quasi-constant features that we are meant to drop using a for loop 
for column in X_training.columns:
    if column not in features_we_keep:
        print(column)

In [0]:
# TASK 5 >>>> Transform X_training 

X_training = selector.transform(X_training)

# TASK 6 >>>> Transform X_testing
X_testing = selector.transform(X_testing)

# Get the shape of X_training and X_testing
X_training.shape, X_testing.shape

In [0]:
# Convert X_training to Pandas DataFrame
X_training = pd.DataFrame(X_training)
# Using the '.columns' attribute assign column names
X_training.columns = features_we_keep

# Convert X_testing to Pandas DataFrame
X_testing = pd.DataFrame(X_training)
# Using the '.columns' attribute assign column names
X_testing.columns = features_we_keep

# Get the first 10 rows of X_train
X_training.head(10)

Find whether our dataset contains duplicated features. You can copy-paste the `for` loop we've already used.

In [0]:
# Find duplicated features in X_training set
features_duplicates = []
for col in range(len(X_training.columns)):
     
    column_1 = X_training.columns[col]
    
    for column_2 in X_training.columns[col + 1:]:
        if X_training[column_1].equals(X_training[column_2]):
            features_duplicates.append(column_2)
            
len(features_duplicates)

In [0]:
# Print the features names
features_duplicates

In [0]:
# TASK 7 >>>> Drop these duplicated features from X_training and X_testing

X_training.drop(labels = features_duplicates, axis = 1, inplace = True)
X_testing.drop(labels = features_duplicates, axis = 1, inplace = True)

In [0]:
# Get the shape of X_training and X_testing
X_training.shape, X_testing.shape

# Apendix

Data sources:

\\(^{1}\\) Santander dataset: https://www.kaggle.com/c/santander-customer-satisfaction/data

\\(^{2}\\) Wine quality dataset: https://archive.ics.uci.edu/ml/datasets/wine+quality

Material adapted for RBI internal purposes with full permissions from original authors.